# Recursion & Recursive Thinking

*Understanding self-referential problem solving*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_02.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand the concept of recursion and how the call stack works
- Identify base cases and recursive cases in recursive functions
- Implement classic recursive algorithms: factorial, Fibonacci, and binary search
- Apply recursion to data structures (lists, strings) and understand its limitations
- Analyze recursive time complexity using recurrence relations and the recursion tree method

---
## 1. What is Recursion?


In the previous week, we explored loops as a way to repeat operations. This week, we study a fundamentally different approach: **recursion**—where a function solves a problem by calling itself with smaller inputs. This technique is not merely a programming trick; it represents a powerful way of thinking about problems that will prove essential for understanding trees, graphs, and divide-and-conquer algorithms.

> 📖 **Definition:** **Recursion** is a problem-solving technique where a function calls itself to solve smaller instances of the same problem. A recursive solution consists of a **base case** (the simplest case, solved directly without recursion) and a **recursive case** (which reduces the problem and calls itself).

The fundamental insight of recursion is that many problems have *self-similar structure*: a problem of size n can be expressed in terms of the same problem of size n-1 (or n/2, or some other smaller size). Once we recognize this pattern, the solution often becomes remarkably elegant.

### Real-World Analogies

Before diving into code, let's build intuition with some real-world examples of recursive thinking:

> 💡 **Analogy 1: Russian Nesting Dolls (Matryoshka)**
>
> Consider finding the smallest doll in a set of nesting dolls:
1. **Base case:** If the doll doesn't open, you've found the smallest.
2. **Recursive case:** If it opens, look inside and repeat the process with the inner doll.
You don't need to know how many dolls there are. The process naturally terminates when you reach one that doesn't open. This is exactly how recursion works in code.

> 💡 **Analogy 2: Looking Up Words in a Dictionary**
>
> When you look up a word, its definition may contain other words you don't know. You look those up too. Eventually, you reach words you already understand—these are your base cases. The process is naturally recursive: understanding a word requires understanding its definition, which may require understanding other words.

> 💡 **Analogy 3: Calculating Your Ancestors**
>
> How many ancestors do you have going back n generations?
- **Base case:** Generation 0 (you) = 1 person
- **Recursive case:** Generation n = 2 × ancestors(n-1)
Each generation doubles: 1, 2, 4, 8, 16... The formula ancestors(n) = 2 × ancestors(n-1) is inherently recursive.

### The Recursive Leap of Faith

When writing recursive code, you must trust that the recursive call returns the correct answer for the smaller problem. This is called the **recursive leap of faith**. If your base case is correct and your recursive case correctly combines the result, the entire solution will be correct—even though it may seem like magic at first.

**Listing 2.1 — The Simplest Recursive Function**

In [ ]:
# A simple countdown using recursion
def countdown(n):
    """Count down from n to 1, then print 'Blastoff!'"""
    if n <= 0:              # Base case: nothing left to count
        print("Blastoff!")
    else:                   # Recursive case
        print(n)
        countdown(n - 1)    # Call ourselves with smaller input

print("Countdown from 5:")
countdown(5)

***Figure 2.1:** A recursive countdown. Each call prints one number, then delegates the rest of the work to a recursive call with a smaller value.*

Notice the structure: we first check if we've reached the base case (n ≤ 0). If not, we do some work (print n), then make a recursive call with a smaller problem (n - 1). Eventually, n becomes 0 and we stop. This pattern appears in virtually all recursive functions.

**Listing 2.2 — Counting Up Recursively**

In [ ]:
# Count from 1 to n recursively
def count_up(n, current=1):
    """Print numbers from current to n."""
    if current > n:         # Base case: passed the target
        return
    print(current)
    count_up(n, current + 1)  # Recursive case: count next number

print("Count up to 5:")
count_up(5)

print("\nCount from 3 to 7:")
count_up(7, 3)

***Figure 2.2:** Recursion can also count upward by passing the current value as a parameter.*

### Why Learn Recursion?

**Natural Fit for Hierarchical Structures:** Trees, graphs, file systems, and nested data have inherently recursive structure. A tree is made of subtrees; a directory contains subdirectories. Processing them recursively often yields the clearest, most natural code.

**Divide and Conquer:** Many efficient algorithms (quicksort, mergesort, binary search) are naturally expressed recursively. They divide the problem into subproblems, solve each recursively, and combine the results.

**Mathematical Elegance:** Mathematical definitions are often recursive (factorial, Fibonacci, combinatorics). Translating them to code becomes almost trivial once you understand recursion.

**Foundation for Advanced Topics:** Understanding recursion is absolutely essential for tree traversals, dynamic programming, backtracking, and graph algorithms covered in later weeks.

---
## 2. The Call Stack


To truly understand recursion, you must understand how function calls work at a deeper level. When Python executes a program, it uses a **call stack** to keep track of function calls and their local state.

> 📖 **Definition:** The **call stack** (or execution stack) is a data structure that stores information about active function calls. Each call creates a **stack frame** (or activation record) containing the function's parameters, local variables, and return address. Frames are added (pushed) when functions are called and removed (popped) when functions return.

The stack follows Last-In-First-Out (LIFO) order: the most recently called function must return before the function that called it can continue. This is why it's called a "stack"—like a stack of plates, you can only add or remove from the top.

**Listing 2.3 — Visualizing the Call Stack**

In [ ]:
# Visualizing recursion with indentation showing stack depth
def factorial(n, depth=0):
    """Calculate n! with call stack visualization."""
    indent = "  " * depth
    print(f"{indent}→ factorial({n}) called")
    
    if n <= 1:  # Base case
        print(f"{indent}← factorial({n}) returns 1")
        return 1
    else:       # Recursive case
        result = n * factorial(n - 1, depth + 1)
        print(f"{indent}← factorial({n}) returns {result}")
        return result

print("Computing factorial(5):")
print("=" * 40)
answer = factorial(5)
print("=" * 40)
print(f"Final answer: 5! = {answer}")

***Figure 2.3:** Arrows show the call stack growing (→) and unwinding (←). The indentation reflects stack depth. Notice how each function waits for its recursive call to return.*

### Stack Frames Step by Step

Let's trace through `factorial(4)` in detail. Each step shows the state of the call stack:

| Step | Action | Stack (bottom → top) |
| --- | --- | --- |
| 1 | Call factorial(4) | [factorial(4)] |
| 2 | 4 ≤ 1 is False, call factorial(3) | [factorial(4), factorial(3)] |
| 3 | 3 ≤ 1 is False, call factorial(2) | [factorial(4), factorial(3), factorial(2)] |
| 4 | 2 ≤ 1 is False, call factorial(1) | [factorial(4), factorial(3), factorial(2), factorial(1)] |
| 5 | 1 ≤ 1 is True, return 1 | [factorial(4), factorial(3), factorial(2)] → returns 1 |
| 6 | factorial(2) computes 2 × 1 = 2, returns | [factorial(4), factorial(3)] → returns 2 |
| 7 | factorial(3) computes 3 × 2 = 6, returns | [factorial(4)] → returns 6 |
| 8 | factorial(4) computes 4 × 6 = 24, returns | [] → returns 24 |

**Listing 2.4 — A More Detailed Stack Trace**

In [ ]:
# Show local variables at each stack frame
def factorial_detailed(n, depth=0):
    """Factorial with detailed state information."""
    indent = "│ " * depth
    print(f"{indent}┌─ ENTER factorial(n={n})")
    
    if n <= 1:
        print(f"{indent}│  Base case reached!")
        print(f"{indent}└─ RETURN 1")
        return 1
    
    print(f"{indent}│  Computing: {n} × factorial({n-1})")
    sub_result = factorial_detailed(n - 1, depth + 1)
    result = n * sub_result
    print(f"{indent}│  Got sub_result = {sub_result}")
    print(f"{indent}│  Computing: {n} × {sub_result} = {result}")
    print(f"{indent}└─ RETURN {result}")
    return result

print("Detailed trace of factorial(4):")
print()
factorial_detailed(4)

***Figure 2.4:** A detailed view showing exactly what happens at each stack frame—entering, computing, waiting for sub-results, and returning.*

### Stack Overflow

The call stack has limited size (typically around 1000 frames in Python). If recursion goes too deep without reaching a base case, you get a `RecursionError`—commonly called a "stack overflow."

**Listing 2.5 — The Recursion Limit**

In [ ]:
import sys

# Python's default recursion limit
limit = sys.getrecursionlimit()
print(f"Python's default recursion limit: {limit}")

# This function has no base case - would crash!
def infinite_recursion(n):
    """BAD: No base case - infinite recursion!"""
    print(f"Call {n}")
    return infinite_recursion(n + 1)  # Never stops!

# Demonstrating what happens with deep (but finite) recursion
def deep_recursion(n, current=0):
    """Count how deep we can go."""
    if current >= n:
        return current
    return deep_recursion(n, current + 1)

# Try progressively deeper recursion
for depth in [100, 500, 900]:
    result = deep_recursion(depth)
    print(f"Successfully reached depth: {result}")

print(f"\n⚠️ Going beyond {limit} would cause RecursionError!")

# You CAN increase the limit (but be careful!):
# sys.setrecursionlimit(2000)

***Figure 2.5:** Python limits recursion depth to prevent crashes. For very deep recursion, consider converting to iteration.*

---
## 3. Anatomy of Recursive Functions


Every well-formed recursive function has exactly two essential components. Missing either one leads to bugs—typically infinite recursion or incorrect results.

### Component 1: Base Case (Termination Condition)

The **base case** handles the simplest possible input directly, without making any recursive calls. It's the "exit condition" that prevents infinite recursion. Common base cases include:

- **Numeric:** n ≤ 0, n == 1, n < 10
- **Collections:** empty list `[]`, empty string `""`, single element
- **Trees:** null node, leaf node
- **Search:** found the target, exhausted search space

### Component 2: Recursive Case (Progress Toward Base)

The **recursive case** handles non-trivial inputs by:

1. Breaking the problem into one or more smaller subproblems
2. Making recursive calls on the subproblems
3. Combining subproblem results into the final answer

Crucially, each recursive call must make **progress toward the base case**. If n starts at 5, each recursive call should use a smaller value (4, 3, 2, 1, 0) until we reach the base case.

**Listing 2.6 — The Recursive Template**

In [ ]:
# Generic template for recursive functions
def recursive_function(problem):
    """
    Template for recursive problem-solving.
    """
    # Step 1: BASE CASE
    # Can we solve this directly without recursion?
    if is_base_case(problem):
        return base_case_solution(problem)
    
    # Step 2: RECURSIVE CASE
    # Break into smaller subproblem(s)
    smaller_problem = make_smaller(problem)
    
    # Step 3: RECURSIVE CALL
    # Trust that this returns the correct answer!
    sub_result = recursive_function(smaller_problem)
    
    # Step 4: COMBINE
    # Build the final answer from sub_result
    return combine(problem, sub_result)

# Concrete example: Sum of 1 + 2 + ... + n
def sum_to_n(n):
    """Calculate 1 + 2 + ... + n recursively."""
    # Base case: sum to 0 is 0
    if n <= 0:
        return 0
    
    # Recursive case: n + sum of (1 to n-1)
    # Trust that sum_to_n(n-1) gives correct answer!
    return n + sum_to_n(n - 1)

# Test
print("Sum from 1 to n:")
for n in [0, 1, 5, 10, 100]:
    result = sum_to_n(n)
    expected = n * (n + 1) // 2  # Formula verification
    status = "✓" if result == expected else "✗"
    print(f"  sum_to_n({n}) = {result} {status}")

***Figure 2.6:** The recursive template applied to sum calculation. Results are verified against the closed-form formula n(n+1)/2.*

### Multiple Base Cases

Some problems require multiple base cases. The Fibonacci sequence is a classic example:

**Listing 2.7 — Multiple Base Cases**

In [ ]:
# Multiple base cases example
def fib(n):
    """
    Fibonacci sequence (naive recursive version).
    fib(0) = 0, fib(1) = 1, fib(n) = fib(n-1) + fib(n-2)
    """
    # TWO base cases needed!
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    # Recursive case: sum of two previous values
    return fib(n - 1) + fib(n - 2)

print("Fibonacci sequence (first 12 numbers):")
for i in range(12):
    print(f"  fib({i}) = {fib(i)}")

***Figure 2.7:** Fibonacci requires two base cases: fib(0) and fib(1). Without both, the recursion would never terminate for n=1.*

### Checklist for Writing Recursive Functions

Before writing a recursive function, answer these questions:

1. **What is the base case?** What's the simplest input? What should it return directly?
2. **What is the recursive case?** How can the problem be expressed in terms of smaller subproblems?
3. **Does each call make progress?** Does each recursive call move strictly closer to the base case?
4. **Do I trust the recursion?** Assume recursive calls return correct results. Does combining them give the right final answer?
5. **Are all cases covered?** Does every possible input eventually reach a base case?

**Listing 2.8 — Applying the Checklist**

In [ ]:
# Problem: Count digits in a positive integer
# 
# Checklist:
# 1. Base case: single digit (n < 10) → return 1
# 2. Recursive case: count_digits(n // 10) + 1
# 3. Progress: n // 10 is smaller than n (removes last digit)
# 4. Trust: count_digits(123 // 10) = count_digits(12) = 2
#           So count_digits(123) = 2 + 1 = 3 ✓
# 5. Coverage: any positive integer eventually becomes < 10

def count_digits(n):
    """Count the number of digits in positive integer n."""
    # Base case: single digit
    if n < 10:
        return 1
    
    # Recursive case: one digit + digits in remaining number
    return 1 + count_digits(n // 10)

# Test with trace
def count_digits_traced(n, depth=0):
    indent = "  " * depth
    print(f"{indent}count_digits({n})")
    
    if n < 10:
        print(f"{indent}→ base case, return 1")
        return 1
    
    sub = count_digits_traced(n // 10, depth + 1)
    result = 1 + sub
    print(f"{indent}→ return 1 + {sub} = {result}")
    return result

print("Trace of count_digits(12345):")
result = count_digits_traced(12345)
print(f"\nFinal result: {result} digits")

***Figure 2.8:** Applying the recursive checklist to count digits. The trace shows how n // 10 makes progress by removing the last digit.*

---
## 4. Classic Example: Factorial


The **factorial** of a non-negative integer n, written n!, is the product of all positive integers from 1 to n:

- 5! = 5 × 4 × 3 × 2 × 1 = 120
- 4! = 4 × 3 × 2 × 1 = 24
- 3! = 3 × 2 × 1 = 6
- 2! = 2 × 1 = 2
- 1! = 1
- 0! = 1 (by mathematical convention)

### Mathematical (Recursive) Definition

Factorial is naturally defined recursively:

- **Base case:** 0! = 1 (and 1! = 1)
- **Recursive case:** n! = n × (n-1)!

This definition translates almost directly to code:

**Listing 2.9 — Factorial: Recursive Implementation**

In [ ]:
# Factorial: Recursive implementation
def factorial_recursive(n):
    """
    Calculate n! recursively.
    
    Time Complexity: O(n) — n recursive calls
    Space Complexity: O(n) — n stack frames
    """
    # Base case
    if n <= 1:
        return 1
    
    # Recursive case: n! = n × (n-1)!
    return n * factorial_recursive(n - 1)

# Test
print("Factorial values:")
for n in range(11):
    print(f"  {n}! = {factorial_recursive(n)}")

***Figure 2.9:** The recursive factorial function directly mirrors the mathematical definition.*

### Trace Table for factorial(5)

Let's trace exactly what happens when we compute 5!:

| Call # | n | Condition | Action | Returns |
| --- | --- | --- | --- | --- |
| 1 | 5 | 5 > 1 | 5 × factorial(4) | 5 × 24 = 120 |
| 2 | 4 | 4 > 1 | 4 × factorial(3) | 4 × 6 = 24 |
| 3 | 3 | 3 > 1 | 3 × factorial(2) | 3 × 2 = 6 |
| 4 | 2 | 2 > 1 | 2 × factorial(1) | 2 × 1 = 2 |
| 5 | 1 | 1 ≤ 1 ✓ | Base case | 1 |

### Comparing Recursive vs Iterative

**Listing 2.10 — Factorial: Iterative Implementation**

In [ ]:
# Factorial: Iterative implementation
def factorial_iterative(n):
    """
    Calculate n! iteratively.
    
    Time Complexity: O(n) — n iterations
    Space Complexity: O(1) — just one variable!
    """
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# Compare both implementations
print("Comparing recursive vs iterative:")
print("-" * 40)
print(f"{'n':<5} {'Recursive':<15} {'Iterative':<15} {'Match'}")
print("-" * 40)

for n in [0, 1, 5, 10, 12]:
    rec = factorial_recursive(n)
    itr = factorial_iterative(n)
    match = "✓" if rec == itr else "✗"
    print(f"{n:<5} {rec:<15} {itr:<15} {match}")

print("-" * 40)
print("Both produce identical results!")
print("But iterative uses O(1) space vs O(n) for recursive.")

***Figure 2.10:** Both implementations compute factorial correctly. The iterative version uses O(1) space, while recursive uses O(n) for the call stack.*

**Listing 2.11 — Factorial: Using Python's Built-in**

In [ ]:
# Python provides a built-in factorial in the math module
import math

# Compare all three implementations
print("All three implementations:")
for n in [0, 1, 5, 10, 20]:
    rec = factorial_recursive(n)
    itr = factorial_iterative(n)
    builtin = math.factorial(n)
    
    all_equal = rec == itr == builtin
    status = "✓ all match" if all_equal else "✗ mismatch!"
    print(f"  {n}! = {builtin} ({status})")

# The built-in is optimized and can handle very large numbers
print(f"\n100! has {len(str(math.factorial(100)))} digits!")
print(f"First 50 chars of 100!: {str(math.factorial(100))[:50]}...")

***Figure 2.11:** For production code, use `math.factorial()`. It's optimized and handles arbitrarily large integers.*

---
## 5. Classic Example: Fibonacci


The **Fibonacci sequence** is one of the most famous sequences in mathematics: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, ... Each number is the sum of the two preceding ones.

### Mathematical Definition

- F(0) = 0
- F(1) = 1
- F(n) = F(n-1) + F(n-2) for n ≥ 2

**Listing 2.12 — Naive Fibonacci (WARNING: Slow!)**

In [ ]:
# Naive recursive Fibonacci - DO NOT use for large n!
call_count = 0

def fib_naive(n):
    """
    Naive Fibonacci implementation.
    
    WARNING: Time Complexity is O(2^n) — EXPONENTIAL!
    Space Complexity: O(n) — stack depth
    """
    global call_count
    call_count += 1
    
    # Base cases
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    # Recursive case
    return fib_naive(n - 1) + fib_naive(n - 2)

# Demonstrate the exponential explosion of calls
print("Naive Fibonacci call counts:")
print("-" * 45)
print(f"{'n':<5} {'fib(n)':<12} {'Calls':<12} {'Growth'}")
print("-" * 45)

prev_calls = 1
for n in [1, 5, 10, 15, 20, 25]:
    call_count = 0
    result = fib_naive(n)
    growth = f"×{call_count / prev_calls:.1f}" if prev_calls > 0 else "-"
    print(f"{n:<5} {result:<12} {call_count:<12} {growth}")
    prev_calls = call_count

print("-" * 45)
print("⚠️ Calls roughly DOUBLE with each increase of 1!")

***Figure 2.12:** Naive Fibonacci is extremely slow. The number of calls roughly doubles with each increment of n, giving O(2^n) time complexity.*

### The Problem: Redundant Computation

The call tree for fib(5) reveals the problem:

> 💡 **Call Tree for fib(5)**
>
> ```

                        fib(5)
                       /      \
                   fib(4)      fib(3)      ← fib(3) computed TWICE
                  /     \      /     \
              fib(3)  fib(2)  fib(2)  fib(1)
              /    \    |
          fib(2)  fib(1) |
            |           |
         fib(1) fib(0)  fib(1) fib(0)

Count of redundant computations:
- fib(3): computed 2 times
- fib(2): computed 3 times  
- fib(1): computed 5 times
- fib(0): computed 3 times
                
```

### Solution 1: Memoization

**Memoization** (not "memorization") is a technique where we cache the results of expensive function calls and return the cached result when the same inputs occur again.

**Listing 2.13 — Fibonacci with Manual Memoization**

In [ ]:
# Fibonacci with manual memoization
def fib_memo(n, cache=None):
    """
    Fibonacci with memoization (caching).
    
    Time Complexity: O(n) — each value computed once
    Space Complexity: O(n) — cache + stack
    """
    # Initialize cache on first call
    if cache is None:
        cache = {}
    
    # Check if already computed
    if n in cache:
        return cache[n]
    
    # Base cases
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    # Compute, cache, and return
    result = fib_memo(n - 1, cache) + fib_memo(n - 2, cache)
    cache[n] = result
    return result

# Test
print("Memoized Fibonacci:")
for n in [10, 20, 30, 40, 50]:
    result = fib_memo(n)
    print(f"  fib({n}) = {result}")

print("\n✓ No exponential slowdown with memoization!")

***Figure 2.13:** Memoization reduces Fibonacci from O(2^n) to O(n) by caching results. Each Fibonacci number is computed only once.*

**Listing 2.14 — Fibonacci with @lru_cache Decorator**

In [ ]:
# Python provides automatic memoization via functools
from functools import lru_cache

@lru_cache(maxsize=None)  # Cache unlimited results
def fib_cached(n):
    """
    Fibonacci with automatic caching via decorator.
    This is the cleanest way to memoize in Python.
    """
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib_cached(n - 1) + fib_cached(n - 2)

# Test with large values
print("Fibonacci with @lru_cache:")
for n in [50, 100, 200, 300]:
    result = fib_cached(n)
    digits = len(str(result))
    print(f"  fib({n}) has {digits} digits")

# Show first 100 digits of fib(300)
big_fib = str(fib_cached(300))
print(f"\nFirst 60 chars of fib(300):")
print(f"  {big_fib[:60]}...")

# Clear cache if needed
# fib_cached.cache_clear()

***Figure 2.14:** The `@lru_cache` decorator from functools provides automatic memoization. This is the recommended approach in Python.*

### Solution 2: Iterative (Best for Fibonacci)

**Listing 2.15 — Fibonacci: Iterative (Optimal)**

In [ ]:
# Fibonacci: Iterative implementation (optimal)
def fib_iterative(n):
    """
    Fibonacci using iteration.
    
    Time Complexity: O(n)
    Space Complexity: O(1) — just two variables!
    """
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    prev, curr = 0, 1
    for _ in range(2, n + 1):
        prev, curr = curr, prev + curr
    return curr

# Compare all implementations
print("Comparing Fibonacci implementations:")
print("-" * 50)

for n in [10, 20, 30]:
    naive = fib_naive(n) if n <= 25 else "too slow"
    memo = fib_memo(n)
    cached = fib_cached(n)
    iterative = fib_iterative(n)
    
    print(f"fib({n}):")
    print(f"  Naive:     {naive}")
    print(f"  Memoized:  {memo}")
    print(f"  Cached:    {cached}")
    print(f"  Iterative: {iterative}")
    print()

print("Verdict: Iterative is best for Fibonacci (O(1) space)")

***Figure 2.15:** The iterative solution is optimal for Fibonacci: O(n) time and O(1) space.*

### Complexity Comparison Table

| Implementation | Time | Space | Notes |
| --- | --- | --- | --- |
| Naive recursive | O(2^n) | O(n) | Never use! Exponential time. |
| Memoized recursive | O(n) | O(n) | Good. Cache uses memory. |
| Iterative | O(n) | O(1) | Best! Minimal memory. |
| Matrix exponentiation | O(log n) | O(1) | Advanced technique. |

---
## 6. Recursion on Data Structures


Recursion naturally processes collections by dividing them into a **head** (first element) and **tail** (remaining elements). This pattern appears constantly in functional programming and is essential for understanding how recursive data structures like linked lists and trees work.

### Sum of a List

**Listing 2.16 — Recursive Sum of List**

In [ ]:
# Sum of list: recursive vs iterative
def sum_recursive(lst):
    """
    Sum all elements in a list recursively.
    
    Base case: empty list → 0
    Recursive case: first element + sum of rest
    """
    if not lst:  # Empty list (base case)
        return 0
    return lst[0] + sum_recursive(lst[1:])

def sum_iterative(lst):
    """Sum all elements iteratively."""
    total = 0
    for x in lst:
        total += x
    return total

# Test with trace
def sum_traced(lst, depth=0):
    indent = "  " * depth
    print(f"{indent}sum({lst})")
    
    if not lst:
        print(f"{indent}→ return 0 (empty)")
        return 0
    
    rest_sum = sum_traced(lst[1:], depth + 1)
    result = lst[0] + rest_sum
    print(f"{indent}→ return {lst[0]} + {rest_sum} = {result}")
    return result

print("Traced sum of [1, 2, 3, 4]:")
print("-" * 40)
total = sum_traced([1, 2, 3, 4])
print("-" * 40)
print(f"Total: {total}")

***Figure 2.16:** The list sum pattern: process the first element, recursively sum the rest, combine by adding.*

### Finding Maximum

**Listing 2.17 — Recursive Maximum**

In [ ]:
# Find maximum element recursively
def find_max(lst):
    """
    Find the maximum element in a non-empty list.
    
    Base case: single element → return it
    Recursive case: max of (first, max of rest)
    """
    if len(lst) == 1:
        return lst[0]
    
    # Get max of the rest
    max_rest = find_max(lst[1:])
    
    # Compare first element with max of rest
    if lst[0] > max_rest:
        return lst[0]
    else:
        return max_rest

# Test
test_lists = [
    [42],
    [3, 1, 4, 1, 5, 9, 2, 6],
    [-5, -2, -8, -1],
    [100, 200, 150, 175]
]

print("Finding maximum recursively:")
for lst in test_lists:
    result = find_max(lst)
    expected = max(lst)  # Verify with built-in
    status = "✓" if result == expected else "✗"
    print(f"  max({lst}) = {result} {status}")

***Figure 2.17:** Finding the maximum by comparing the first element with the maximum of the rest.*

### Checking Membership

**Listing 2.18 — Recursive Membership Check**

In [ ]:
# Check if element exists in list
def contains(lst, target):
    """
    Check if target is in lst (recursive version of 'in').
    
    Base case 1: empty list → False
    Base case 2: first element matches → True
    Recursive case: check rest of list
    """
    if not lst:
        return False
    if lst[0] == target:
        return True
    return contains(lst[1:], target)

# Test
numbers = [10, 20, 30, 40, 50]
print(f"List: {numbers}")
print()

for target in [30, 35, 10, 50, 0]:
    result = contains(numbers, target)
    expected = target in numbers
    status = "✓" if result == expected else "✗"
    print(f"  contains({target}): {result} {status}")

***Figure 2.18:** Membership check with multiple base cases: found (True) or exhausted (False).*

### Flattening Nested Lists

**Listing 2.19 — Flattening Nested Structures**

In [ ]:
# Flatten arbitrarily nested lists
def flatten(nested):
    """
    Flatten nested lists into a single flat list.
    
    Example: [1, [2, [3, 4], 5], 6] → [1, 2, 3, 4, 5, 6]
    """
    result = []
    for item in nested:
        if isinstance(item, list):
            # Item is a list: recursively flatten it
            result.extend(flatten(item))
        else:
            # Item is not a list: add directly
            result.append(item)
    return result

# Test with various nesting levels
test_cases = [
    [1, 2, 3],                           # Already flat
    [1, [2, 3], 4],                       # One level
    [1, [2, [3, [4, [5]]]]],              # Deep nesting
    [[[[1]]], [[[2]]], [[[3]]]],          # Very nested
    [1, [2, 3], [4, [5, 6]], 7, [8]],     # Mixed
]

print("Flattening nested lists:")
for nested in test_cases:
    flat = flatten(nested)
    print(f"  {nested}")
    print(f"  → {flat}")
    print()

***Figure 2.19:** Flattening handles arbitrary nesting depth by recursing whenever we encounter a sublist.*

### Binary Search

**Listing 2.20 — Recursive Binary Search**

In [ ]:
# Binary search: recursive implementation
def binary_search(arr, target, low=0, high=None):
    """
    Search for target in sorted array using binary search.
    
    Returns index if found, -1 if not found.
    Time: O(log n) — halves search space each call
    Space: O(log n) — stack depth
    """
    if high is None:
        high = len(arr) - 1
    
    # Base case: empty range
    if low > high:
        return -1
    
    mid = (low + high) // 2
    
    if arr[mid] == target:
        return mid                    # Found it!
    elif arr[mid] > target:
        return binary_search(arr, target, low, mid - 1)    # Search left
    else:
        return binary_search(arr, target, mid + 1, high)   # Search right

# Test
sorted_arr = [2, 5, 8, 12, 16, 23, 38, 56, 72, 91]
print(f"Sorted array: {sorted_arr}")
print()

for target in [23, 2, 91, 50, 0, 100]:
    idx = binary_search(sorted_arr, target)
    if idx != -1:
        print(f"  binary_search({target}) = index {idx} (arr[{idx}] = {sorted_arr[idx]})")
    else:
        print(f"  binary_search({target}) = -1 (not found)")

***Figure 2.20:** Binary search recursively halves the search space, achieving O(log n) time complexity.*

---
## 7. Recursion on Strings


Strings work like lists for recursion: process the first character, then recurse on the rest. Python's string slicing makes this natural.

### String Reversal

**Listing 2.21 — Recursive String Reversal**

In [ ]:
# Reverse a string recursively
def reverse_string(s):
    """
    Reverse a string recursively.
    
    Base case: empty or single char → return as-is
    Recursive case: reverse(rest) + first_char
    """
    if len(s) <= 1:
        return s
    return reverse_string(s[1:]) + s[0]

# Test
test_strings = ["", "a", "ab", "hello", "Python", "racecar"]

print("String reversal:")
for s in test_strings:
    rev = reverse_string(s)
    expected = s[::-1]  # Verify with slicing
    status = "✓" if rev == expected else "✗"
    print(f"  '{s}' → '{rev}' {status}")

***Figure 2.21:** String reversal by moving the first character to the end of the reversed rest.*

### Palindrome Check

**Listing 2.22 — Recursive Palindrome Check**

In [ ]:
# Check if string is a palindrome
def is_palindrome(s):
    """
    Check if s reads the same forwards and backwards.
    Ignores case and spaces for natural text.
    """
    # Normalize: lowercase, no spaces
    s = s.lower().replace(" ", "")
    
    # Base case: empty or single char is palindrome
    if len(s) <= 1:
        return True
    
    # Check first and last characters
    if s[0] != s[-1]:
        return False
    
    # Recurse on middle portion
    return is_palindrome(s[1:-1])

# Test
test_cases = [
    ("racecar", True),
    ("hello", False),
    ("A man a plan a canal Panama", True),
    ("Was it a car or a cat I saw", True),
    ("Python", False),
    ("", True),
    ("a", True),
]

print("Palindrome check:")
for s, expected in test_cases:
    result = is_palindrome(s)
    status = "✓" if result == expected else "✗"
    print(f"  '{s}' → {result} {status}")

***Figure 2.22:** Palindrome check compares first and last characters, then recurses on the middle.*

### Count Character Occurrences

**Listing 2.23 — Count Characters Recursively**

In [ ]:
# Count occurrences of a character in a string
def count_char(s, char):
    """
    Count how many times char appears in s.
    """
    if not s:
        return 0
    
    # Count 1 if first char matches, else 0
    count = 1 if s[0] == char else 0
    
    # Add count from rest of string
    return count + count_char(s[1:], char)

# Test
text = "mississippi"
print(f"String: '{text}'")
print()

for char in ['m', 'i', 's', 'p', 'x']:
    result = count_char(text, char)
    expected = text.count(char)  # Verify with built-in
    status = "✓" if result == expected else "✗"
    print(f"  count('{char}'): {result} {status}")

***Figure 2.23:** Counting characters by checking each position and summing recursively.*

### String Length Without len()

**Listing 2.24 — Recursive String Length**

In [ ]:
# Calculate string length without using len()
def string_length(s):
    """
    Calculate the length of a string recursively.
    
    Base case: empty string → 0
    Recursive case: 1 + length of rest
    """
    if s == "":
        return 0
    return 1 + string_length(s[1:])

# Test
test_strings = ["", "a", "hello", "Python programming", "12345"]

print("String length (recursive):")
for s in test_strings:
    result = string_length(s)
    expected = len(s)
    status = "✓" if result == expected else "✗"
    print(f"  len('{s}') = {result} {status}")

***Figure 2.24:** Counting length by adding 1 for each character until we reach the empty string.*

---
## 8. Analyzing Recursive Complexity


Analyzing recursive algorithms requires understanding **recurrence relations**—equations that express the cost of a computation in terms of the cost of smaller subcomputations.

### Common Recurrence Patterns

| Recurrence | Solution | Example |
| --- | --- | --- |
| T(n) = T(n-1) + O(1) | O(n) | Factorial, list sum |
| T(n) = T(n-1) + O(n) | O(n²) | Selection sort |
| T(n) = T(n/2) + O(1) | O(log n) | Binary search |
| T(n) = T(n/2) + O(n) | O(n) | Finding median |
| T(n) = 2T(n/2) + O(1) | O(n) | Tree traversal |
| T(n) = 2T(n/2) + O(n) | O(n log n) | Merge sort |
| T(n) = T(n-1) + T(n-2) | O(2^n) | Naive Fibonacci |
| T(n) = 2T(n-1) | O(2^n) | Power set, permutations |

**Listing 2.25 — Counting Recursive Calls**

In [ ]:
# Count calls to verify complexity analysis

# O(n) - Linear recursion
def linear_calls(n, counter):
    counter['calls'] += 1
    if n <= 0:
        return
    linear_calls(n - 1, counter)

# O(log n) - Halving recursion
def log_calls(n, counter):
    counter['calls'] += 1
    if n <= 1:
        return
    log_calls(n // 2, counter)

# O(2^n) - Exponential recursion
def exp_calls(n, counter):
    counter['calls'] += 1
    if n <= 0:
        return
    exp_calls(n - 1, counter)
    exp_calls(n - 1, counter)

print("Call counts by recursion type:")
print("-" * 50)
print(f"{'n':<8} {'Linear O(n)':<15} {'Log O(log n)':<15} {'Exp O(2^n)'}")
print("-" * 50)

for n in [5, 10, 15, 20]:
    c1, c2, c3 = {'calls': 0}, {'calls': 0}, {'calls': 0}
    linear_calls(n, c1)
    log_calls(n, c2)
    if n <= 20:
        exp_calls(n, c3)
        exp_str = str(c3['calls'])
    else:
        exp_str = "too many"
    
    print(f"{n:<8} {c1['calls']:<15} {c2['calls']:<15} {exp_str}")

print("-" * 50)

***Figure 2.25:** Empirical verification of time complexity by counting recursive calls.*

### Space Complexity

Recursive space complexity is determined by the **maximum depth** of the call stack multiplied by the space per frame. Even exponential-time algorithms like naive Fibonacci have only O(n) space because the stack never gets deeper than n.

**Listing 2.26 — Measuring Stack Depth**

In [ ]:
# Track maximum stack depth
def track_depth(n, current=1, max_depth=None):
    if max_depth is None:
        max_depth = [0]
    
    max_depth[0] = max(max_depth[0], current)
    
    if n <= 1:
        return max_depth[0]
    
    return track_depth(n - 1, current + 1, max_depth)

# For binary recursion, depth is still O(n), not O(2^n)
def binary_depth(n, current=1, max_depth=None):
    if max_depth is None:
        max_depth = [0]
    
    max_depth[0] = max(max_depth[0], current)
    
    if n <= 0:
        return max_depth[0]
    
    # Even with two recursive calls, only one path is active at a time
    binary_depth(n - 1, current + 1, max_depth)
    binary_depth(n - 1, current + 1, max_depth)
    
    return max_depth[0]

print("Stack depth analysis:")
print("-" * 40)
print(f"{'n':<10} {'Linear':<15} {'Binary'}")
print("-" * 40)

for n in [5, 10, 15, 20]:
    linear = track_depth(n)
    binary = binary_depth(n)
    print(f"{n:<10} {linear:<15} {binary}")

print("-" * 40)
print("Note: Binary recursion depth = n, NOT 2^n!")
print("Only one branch is active at any time.")

***Figure 2.26:** Stack depth for binary recursion is still O(n) because only one path through the call tree is active at any moment.*

---
## 9. Recursion vs Iteration


Any recursive algorithm can be converted to iteration, and vice versa. The choice depends on clarity, efficiency, and the nature of the problem.

### When to Use Recursion

- **Tree/graph structures:** Traversal is naturally recursive
- **Divide and conquer:** Mergesort, quicksort, binary search
- **Mathematical definitions:** Factorial, Fibonacci, combinations
- **Nested structures:** Parsing, flattening, XML/JSON processing
- **Backtracking:** Puzzles, constraint satisfaction

### When to Use Iteration

- **Simple loops:** Summing, counting, transforming sequences
- **Performance critical:** Avoiding function call overhead
- **Deep recursion:** When stack overflow is a risk
- **Tail recursion:** (Python doesn't optimize this anyway)

**Listing 2.27 — Converting Recursion to Iteration**

In [ ]:
# Converting between recursion and iteration

# Example 1: Factorial
def fact_recursive(n):
    if n <= 1:
        return 1
    return n * fact_recursive(n - 1)

def fact_iterative(n):
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# Example 2: Sum of list
def sum_recursive(lst):
    if not lst:
        return 0
    return lst[0] + sum_recursive(lst[1:])

def sum_iterative(lst):
    total = 0
    for x in lst:
        total += x
    return total

# Example 3: Reverse string
def rev_recursive(s):
    if len(s) <= 1:
        return s
    return rev_recursive(s[1:]) + s[0]

def rev_iterative(s):
    result = ""
    for char in s:
        result = char + result
    return result

# Test all pairs
print("Recursive vs Iterative:")
print("-" * 50)

# Factorial
print(f"factorial(10):")
print(f"  Recursive: {fact_recursive(10)}")
print(f"  Iterative: {fact_iterative(10)}")

# Sum
lst = [1, 2, 3, 4, 5]
print(f"\nsum({lst}):")
print(f"  Recursive: {sum_recursive(lst)}")
print(f"  Iterative: {sum_iterative(lst)}")

# Reverse
s = "Python"
print(f"\nreverse('{s}'):")
print(f"  Recursive: '{rev_recursive(s)}'")
print(f"  Iterative: '{rev_iterative(s)}'")

***Figure 2.27:** Side-by-side comparison of recursive and iterative implementations.*

### Using an Explicit Stack

Any recursion can be converted to iteration using an explicit stack data structure:

**Listing 2.28 — Explicit Stack for Recursion**

In [ ]:
# Converting recursion to iteration with explicit stack

# Factorial with explicit stack
def fact_explicit_stack(n):
    """Factorial using explicit stack (simulating call stack)."""
    stack = []
    
    # Push phase: simulate recursive descent
    while n > 1:
        stack.append(n)
        n -= 1
    
    # Pop phase: simulate returns
    result = 1
    while stack:
        result *= stack.pop()
    
    return result

# Test
print("Factorial with explicit stack:")
for n in [1, 5, 10]:
    result = fact_explicit_stack(n)
    expected = fact_recursive(n)
    status = "✓" if result == expected else "✗"
    print(f"  {n}! = {result} {status}")

# This technique is essential for tree traversal (Week 6)
# where we might need iterative versions to avoid stack overflow

***Figure 2.28:** Using an explicit stack to simulate recursion. This technique becomes important for tree and graph algorithms.*

| Aspect | Recursion | Iteration |
| --- | --- | --- |
| Code clarity | Often cleaner for recursive problems | May be verbose for trees/graphs |
| Space | O(depth) stack frames | O(1) or explicit stack on heap |
| Function overhead | Higher (call/return cost) | Lower (no call overhead) |
| Stack overflow risk | Yes, for deep recursion | No (if using heap for stack) |
| Debugging | Stack trace helpful | May need to track state manually |

---
## 10. Common Pitfalls


### Pitfall 1: Missing Base Case

**Listing 2.29 — Missing Base Case**

In [ ]:
# BAD: No base case - infinite recursion!
def bad_countdown(n):
    print(n)
    bad_countdown(n - 1)  # Never stops!

# GOOD: Has base case
def good_countdown(n):
    if n <= 0:            # BASE CASE!
        print("Blastoff!")
        return
    print(n)
    good_countdown(n - 1)

print("Good countdown:")
good_countdown(3)
print("\nbad_countdown would run forever (RecursionError)")

***Figure 2.29:** Always include a base case to prevent infinite recursion.*

### Pitfall 2: Not Making Progress

**Listing 2.30 — Not Making Progress**

In [ ]:
# BAD: Recursive call doesn't make progress
def bad_sum(lst):
    if not lst:
        return 0
    return lst[0] + bad_sum(lst)  # WRONG! Should be lst[1:]

# GOOD: Each call processes smaller list
def good_sum(lst):
    if not lst:
        return 0
    return lst[0] + good_sum(lst[1:])  # lst[1:] is smaller

print("Good sum:")
print(f"  sum([1,2,3,4,5]) = {good_sum([1,2,3,4,5])}")
print("\nbad_sum would recurse forever on the same list!")

***Figure 2.30:** Each recursive call must make progress toward the base case.*

### Pitfall 3: Wrong Base Case

**Listing 2.31 — Wrong Base Case**

In [ ]:
# BAD: Base case doesn't cover all terminating inputs
def bad_factorial(n):
    if n == 1:          # What about n=0?
        return 1
    return n * bad_factorial(n - 1)

# GOOD: Base case handles all valid terminating inputs
def good_factorial(n):
    if n <= 1:          # Handles both 0 and 1
        return 1
    return n * good_factorial(n - 1)

print("Factorial edge cases:")
print(f"  good_factorial(0) = {good_factorial(0)}")
print(f"  good_factorial(1) = {good_factorial(1)}")
print(f"  good_factorial(5) = {good_factorial(5)}")
print("\nbad_factorial(0) would cause infinite recursion!")

***Figure 2.31:** Base cases must handle all valid terminating inputs.*

### Pitfall 4: Forgetting to Return

**Listing 2.32 — Forgetting to Return**

In [ ]:
# BAD: Forgetting to return the recursive result
def bad_factorial(n):
    if n <= 1:
        return 1
    bad_factorial(n - 1) * n  # Missing return!

# GOOD: Return the result
def good_factorial(n):
    if n <= 1:
        return 1
    return n * good_factorial(n - 1)  # Don't forget return!

print("Return values:")
print(f"  good_factorial(5) = {good_factorial(5)}")
try:
    result = bad_factorial(5)
    print(f"  bad_factorial(5) = {result}")
except TypeError as e:
    print(f"  bad_factorial(5) crashed: {e}")
print()
print("Without return, the recursive call's result is lost.")
print("Python returns None by default → None * n → TypeError!")

***Figure 2.32:** Always return the result of recursive calls. Forgetting `return` gives `None`.*

- **Recursion** solves problems by breaking them into smaller versions of themselves
- Every recursive function needs a **base case** and a **recursive case**
- The **call stack** tracks active function calls; deep recursion can overflow it
- **Memoization** eliminates redundant computation (e.g., Fibonacci O(2^n) → O(n))
- Common recurrences: T(n-1)+O(1)=O(n), T(n/2)+O(1)=O(log n), 2T(n/2)+O(n)=O(n log n)
- Recursion and iteration are interchangeable; choose based on clarity and efficiency

---
# 📝 Exercises


### Exercise 1: Power Function  (⭐ Easy)

Write `power(base, exp)` that calculates base^exp recursively without using the `**` operator. Assume exp ≥ 0.

**Expected:** (Expected: power(2, 10) = 1024, power(3, 4) = 81, power(5, 0) = 1)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Base Case:**

                        Any number raised to the power of 0 equals 1.

`if exp == 0: return 1`
- **Hint 2 - Recursive Case:**

`base^exp = base × base^(exp-1)`

                        Example: 2³ = 2 × 2² = 2 × 2 × 2¹ = 2 × 2 × 2 × 2⁰
- **Hint 3 - Structure:**

`return base * power(base, exp - 1)`
</details>

In [ ]:
# ✏️ [EX1]
# Power Function - Calculate base^exp recursively

def power(base, exp):
    # Your code here
    pass

# Test your function (uncomment these lines)
# print(f"power(2, 10) = {power(2, 10)}")   # Expected: 1024
# print(f"power(3, 4) = {power(3, 4)}")     # Expected: 81
# print(f"power(5, 0) = {power(5, 0)}")     # Expected: 1
# print(f"power(7, 1) = {power(7, 1)}")     # Expected: 7

### Exercise 2: Sum of Digits  (⭐ Easy)

Write `digit_sum(n)` that returns the sum of all digits in a positive integer n.

**Expected:** (Expected: digit_sum(12345) = 15, digit_sum(9999) = 36, digit_sum(7) = 7)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Extract Last Digit:**

                        Use modulo to get the last digit: `n % 10`

                        Example: 12345 % 10 = 5
- **Hint 2 - Remove Last Digit:**

                        Use integer division: `n // 10`

                        Example: 12345 // 10 = 1234
- **Hint 3 - Base Case:**

                        When n is a single digit (n < 10), return n itself.

`digit_sum(12345) = 5 + digit_sum(1234)`
</details>

In [ ]:
# ✏️ [EX2]
# Sum of Digits - Return the sum of all digits in n

def digit_sum(n):
    # Your code here
    pass

# Test your function (uncomment these lines)
# print(f"digit_sum(12345) = {digit_sum(12345)}")  # Expected: 15
# print(f"digit_sum(9999) = {digit_sum(9999)}")    # Expected: 36
# print(f"digit_sum(7) = {digit_sum(7)}")          # Expected: 7
# print(f"digit_sum(100) = {digit_sum(100)}")      # Expected: 1

### Exercise 3: GCD (Euclid's Algorithm)  (⭐⭐ Medium)

Write `gcd(a, b)` to find the greatest common divisor using Euclid's algorithm: gcd(a, b) = gcd(b, a % b), with base case gcd(a, 0) = a.

**Expected:** (Expected: gcd(48, 18) = 6, gcd(100, 25) = 25, gcd(17, 13) = 1)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Base Case:**

                        When b equals 0, return a.

`if b == 0: return a`
- **Hint 2 - Euclid's Formula:**

`gcd(a, b) = gcd(b, a % b)`

                        The remainder replaces the larger number.
- **Hint 3 - Trace Example:**

                        gcd(48, 18) → gcd(18, 12) → gcd(12, 6) → gcd(6, 0) = 6
</details>

In [ ]:
# ✏️ [EX3]
# GCD using Euclid's Algorithm

def gcd(a, b):
    # Your code here
    pass

# Test your function (uncomment these lines)
# print(f"gcd(48, 18) = {gcd(48, 18)}")    # Expected: 6
# print(f"gcd(100, 25) = {gcd(100, 25)}")  # Expected: 25
# print(f"gcd(17, 13) = {gcd(17, 13)}")    # Expected: 1
# print(f"gcd(54, 24) = {gcd(54, 24)}")    # Expected: 6

### Exercise 4: Count Occurrences in List  (⭐⭐ Medium)

Write `count_occurrences(lst, target)` that counts how many times target appears in lst, recursively (without using `.count()`).

**Expected:** (Expected: count_occurrences([1,2,3,2,4,2], 2) = 3, count_occurrences([1,1,1,1], 1) = 4)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Base Case:**

                        Empty list has 0 occurrences.

`if not lst: return 0`
- **Hint 2 - Check First Element:**

                        If `lst[0] == target`, add 1; otherwise add 0.

                        Use a conditional expression: `(1 if lst[0] == target else 0)`
- **Hint 3 - Recurse on Rest:**

                        Add the count from the rest of the list: `lst[1:]`

`return (match) + count_occurrences(lst[1:], target)`
</details>

In [ ]:
# ✏️ [EX4]
# Count Occurrences - Count target appearances in list recursively

def count_occurrences(lst, target):
    # Your code here
    pass

# Test your function (uncomment these lines)
# print(f"count_occurrences([1,2,3,2,4,2], 2) = {count_occurrences([1,2,3,2,4,2], 2)}")  # Expected: 3
# print(f"count_occurrences([1,1,1,1], 1) = {count_occurrences([1,1,1,1], 1)}")          # Expected: 4
# print(f"count_occurrences([1,2,3], 5) = {count_occurrences([1,2,3], 5)}")              # Expected: 0
# print(f"count_occurrences([], 1) = {count_occurrences([], 1)}")                        # Expected: 0

### Exercise 5: Tower of Hanoi  (⭐⭐⭐ Hard)

Write `hanoi(n, source, target, auxiliary)` that prints the moves needed to solve the Tower of Hanoi puzzle with n disks.

**Expected:** (Expected: hanoi(3, 'A', 'C', 'B') prints 7 moves, hanoi(4, ...) prints 15 moves)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Base Case:**

                        When n == 1, just move the single disk directly:

`print(f"Move disk from {source} to {target}")`
- **Hint 2 - The Three-Step Strategy:**

                        To move n disks from A to C using B:

                        1. Move n-1 disks from A to B (using C)

                        2. Move disk n from A to C

                        3. Move n-1 disks from B to C (using A)
- **Hint 3 - Recursive Calls:**

`hanoi(n-1, source, auxiliary, target)` - step 1

`print(move)` - step 2

`hanoi(n-1, auxiliary, target, source)` - step 3
</details>

In [ ]:
# ✏️ [EX5]
# Tower of Hanoi: Move n disks from source to target using auxiliary

def hanoi(n, source, target, auxiliary):
    # Your code here
    pass

# Test your function (uncomment these lines)
# print("Tower of Hanoi with 3 disks:")
# hanoi(3, 'A', 'C', 'B')

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request
import time as _time, datetime as _dt

WEEK = "Week_02"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': len(_code) > 5
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': len(_code) > 5
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'dsa-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
